In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_log_error

# X_train = pd.read_csv("new_train.csv")
# X_test = pd.read_csv("new_test.csv")
X = pd.read_csv("new_train_1h.csv")
test_df = pd.read_csv("new_test_1h.csv")

In [2]:
train_df = X.sample(frac=0.6,random_state=200)
evaluate_df = X.drop(train_df.index)

X_train = train_df.drop(['Id','SalePrice'], axis=1)
Y_train = train_df["SalePrice"]
X_train_full = X.drop(['Id','SalePrice'], axis=1)
Y_train_full = X["SalePrice"]
X_eval  = evaluate_df.drop(['Id','SalePrice'], axis=1)
Y_eval  = evaluate_df["SalePrice"]
X_test = test_df.drop('Id', axis=1)

In [3]:
# X_train_new = X_train[['Id','TotalSquareFeet', 'NeighborhoodOverallQualNCond', 'OverallQual', \
#            'ExterQual', 'NeighborhoodBldgType', 'GarageCars', 'TotRmsAbvGrd', \
#            'GrLivArea', 'TotalBsmtSF', 'SalePrice']]
# X_train_new.head(20)

In [4]:
# X_test_new = X_test[['Id','TotalSquareFeet', 'NeighborhoodOverallQualNCond', 'OverallQual', \
#            'ExterQual', 'NeighborhoodBldgType', 'GarageCars', 'TotRmsAbvGrd', \
#            'GrLivArea', 'TotalBsmtSF']]
# X_test_new.head(20)

In [5]:
# X_train_new.to_csv('new_train_limit.csv', index=False)
# X_test_new.to_csv('new_test_limit.csv', index=False)
# print("New files have been created.")

In [6]:
len(X_train_full.columns)

104

In [7]:
#from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb

# grad_boost = GradientBoostingRegressor(random_state=0, n_estimators=500, max_features= 0.3)
# grad_boost_full = GradientBoostingRegressor(random_state=0, n_estimators=500, max_features= 0.3)
grad_boost = xgb.XGBRegressor( booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=5, min_child_weight=1.5, n_estimators=1900, 
             n_jobs=1, nthread=None, objective='reg:squarederror',
             reg_alpha=0.6, reg_lambda=0.6, scale_pos_weight=1, 
             silent=None, subsample=0.8, verbosity=1)
grad_boost_full = xgb.XGBRegressor( booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=5, min_child_weight=1.5, n_estimators=1900, 
             n_jobs=1, nthread=None, objective='reg:squarederror',
             reg_alpha=0.6, reg_lambda=0.6, scale_pos_weight=1, 
             silent=None, subsample=0.8, verbosity=1)
num_feat_to_drop = 1
dropped_features = []
i = 0
best_acc = 1

while len(X_train.columns) > 5:
    # Train our model with the set
    grad_boost.fit(X_train, Y_train)
    grad_boost_full.fit(X_train_full, Y_train_full)
    
    # Determine our accurancy with this model
    acc_train = np.sqrt(mean_squared_log_error(Y_train, grad_boost.predict(X_train)))
    acc_eval = np.sqrt(mean_squared_log_error(Y_eval, grad_boost.predict(X_eval)))
    acc_full = np.sqrt(mean_squared_log_error(Y_train_full, grad_boost_full.predict(X_train_full)))
    avg_acc = (acc_train + acc_eval + acc_full) / 3
    if (avg_acc >= best_acc):
        print (f'{i}: Train = {acc_train}  Evaluate = {acc_eval} Full = {acc_full} Average = {avg_acc}')
    else:
        best_acc = avg_acc
        print (f'{i}: Train = {acc_train}  Evaluate = {acc_eval} Full = {acc_full} Average = {avg_acc} - NEW BEST!!!')
        print (f'  Dropped: {dropped_features}')
    
    # Determine the least important features
    importances = pd.DataFrame({'feature':X_train.columns,'importance':np.round(grad_boost.feature_importances_,3)})
    importances = importances.sort_values('importance',ascending=True).set_index('feature')
    
    # Drop the X least important features
    features_to_drop = importances[0:num_feat_to_drop].index.tolist()
    if len(features_to_drop) <= 0:
        break
    X_train.drop(features_to_drop, axis=1, inplace=True)
    X_eval.drop(features_to_drop, axis=1, inplace=True)
    X_train_full.drop(features_to_drop, axis=1, inplace=True)
    dropped_features.append(features_to_drop)
    
    i += num_feat_to_drop

0: Train = 0.02160843987064369  Evaluate = 0.1332962759446923 Full = 0.03474838513463686 Average = 0.06321770031665762 - NEW BEST!!!
  Dropped: []
1: Train = 0.021792834156171496  Evaluate = 0.13236501452758254 Full = 0.0349216956149938 Average = 0.06302651476624928 - NEW BEST!!!
  Dropped: [['LotConfig_4']]
2: Train = 0.02163909737885441  Evaluate = 0.13290614091330422 Full = 0.03458922651207918 Average = 0.0630448216014126
3: Train = 0.021368133617347216  Evaluate = 0.1331646204557433 Full = 0.034403921121681544 Average = 0.06297889173159067 - NEW BEST!!!
  Dropped: [['LotConfig_4'], ['Foundation_3'], ['Neighborhood_3']]
4: Train = 0.021523511687122302  Evaluate = 0.13283964145832042 Full = 0.0345520888800066 Average = 0.06297174734181644 - NEW BEST!!!
  Dropped: [['LotConfig_4'], ['Foundation_3'], ['Neighborhood_3'], ['BldgType_2']]
5: Train = 0.021521506537900203  Evaluate = 0.13404810310845522 Full = 0.03449135236385331 Average = 0.06335365400340291
6: Train = 0.021412523418256862

KeyboardInterrupt: 

In [8]:
best_dropped = [['LotConfig_4'], ['Foundation_3'], ['Neighborhood_3'], ['BldgType_2'], ['Neighborhood_21'], ['Neighborhood_12'], ['Neighborhood_8'], ['HouseStyle_8'], ['BldgType_3'], ['HouseStyle_2'], ['Neighborhood_10']]
best_dropped = [x for inner in best_dropped for x in inner]
best_dropped

['LotConfig_4',
 'Foundation_3',
 'Neighborhood_3',
 'BldgType_2',
 'Neighborhood_21',
 'Neighborhood_12',
 'Neighborhood_8',
 'HouseStyle_8',
 'BldgType_3',
 'HouseStyle_2',
 'Neighborhood_10']